In [9]:
import panel as pn, pandas as pd, matplotlib, bokeh, holoviews as hv, numpy as np
import panel.widgets as pnw
import hvplot.pandas

In [50]:
df = pd.read_csv('covid_impact_on_airport_traffic.csv')
df['Date'] = df.Date.astype('datetime64[ns]')
df_full = df[['Date','AirportName','PercentOfBaseline', 'City', 'State', 'Centroid']].copy()
#df = df_full[df_full['AirportName'].isin(['Seattle-Tacoma International ', 'Newark Liberty International ','LaGuardia', 'John F. Kennedy International'])].copy()
dfp = pd.pivot(data=df, index='Date', columns='AirportName', values='PercentOfBaseline')

In [6]:
def mplot(avg):
    ax = Figure().add_subplot()
    avg.plot(ax=ax)
    return ax.figure

def fn(airport='Seattle-Tacoma International ', days=30, view_fn=mplot ):
    avg = dfp[airport].rolling(window=days).mean()
    return view_fn(avg)

In [7]:
hover = hv.streams.PointerX(x=dfp
                            .index.min())

def hvplot(avg, highlight):
    return avg.hvplot(height=200) * highlight.hvplot.scatter(color='orange', padding=0.1)

def hlplot2(avg):
    line = avg.hvplot(responsive=True, height=300, legend=False)
    hover.source = line
    return line

def table(x):
    index = np.abs((dfp.index - x).astype(int)).argmin()
    return dfp.iloc[index]

In [58]:
title = "## Airport Traffic Compared to Pre-Covid Levels"
description = "Each day, the value of the air traffic is listed as a prorportion of the traffic from the same day of the week pre-Covid. \
The averages are then calculated based on the time window selected below."

ap = pnw.Select(name='Airport', value='Seattle-Tacoma International ', 
                             options=list(dfp.columns), size=1, width=250)
days = pnw.IntSlider(name='Rolling Average Window', value=30, start=1, end=90,
                         sizing_mode='stretch_width')

contents = pn.Row(
    pn.Column(description, ap, days, width=260),
    pn.bind(fn, ap, days, hlplot2),
    pn.panel(pn.bind(table, hover.param.x)),
    width_policy='max')

app = pn.Column(
       title, 
       contents,
       width_policy='max')

In [60]:
#app.show()

In [ ]:
app.servable('Airport Traffic Changes');

In [29]:
#template = pn.template.MaterialTemplate(title='Airport Traffic Compared to Pre-Covid Levels')
#template.sidebar.extend([description, ap, days])
#template.main.append(contents)

In [30]:
#template.show()

Launching server at http://localhost:56911
